# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
#import random
from scipy.stats import linregress
from datetime import date


# Import API key
from api_keys import weather_api_key



In [2]:
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

601

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
#Building the url components that will be statis throughout the search
base_url = "http://api.openweathermap.org/data/2.5/weather?"

params = {
    "units": "imperial",
    "appid": weather_api_key,
}

In [5]:
def convert_temp (temp, scale='C', to='F'):
    '''
    Takes a temperature and converts it to a different scale
    temp must be numeric
    valid values form the starting scale and to scale are 'C': Celsius, 'F': Farenheit, and 'K': Kelvin
    '''
    scales = ['C','F','K']
    #print(scale.upper(),'\n',to,'\n',scales)
    if not ((scale.upper() in scales) and (to.upper() in scales)):
        return ValueError("valid values form the starting scale and to scale are 'C': Celsius, 'F': Farenheit, and 'K': Kelvin")
    
    if scale == 'C' and to == 'F':
        return 32.0 + ((9.0 / 5) * temp)
    
    if scale == 'F' and to == 'C':
        return (-32.0 + temp) * (5.0 / 9)
    
    if scale == 'K' and to == 'C':
        return temp - 273.15
    
    if scale == 'C' and to == 'K':
        return temp + 273.15
    
    if scale == 'K' and to == 'F':
        return 32.0 + ((9.0 / 5) * (temp - 273.15))
    
    if scale == 'F' and to == 'K':
        return ((-32.0 + temp) * (5.0 / 9)) + 273.15
    
    
convert_temp(0)

#Note: this function is not used

32.0

In [6]:
#figuring out which data to extract for each city
params["q"] = random.choice(cities)

#response = requests.get(base_url, params=params)
'''
{'coord': {'lon': -117.16, 'lat': 32.72},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 16.93,
  'feels_like': 17.09,
  'temp_min': 15,
  'temp_max': 20,
  'pressure': 1013,
  'humidity': 77},
 'visibility': 14484,
 'wind': {'speed': 1.05, 'deg': 257},
 'clouds': {'all': 90},
 'dt': 1594289893,
 'sys': {'type': 1,
  'id': 5771,
  'country': 'US',
  'sunrise': 1594298899,
  'sunset': 1594349950},
 'timezone': -25200,
 'id': 5391811,
 'name': 'San Diego',
 'cod': 200}
'''

'''
City          550
Lat           550
Lng           550
Max Temp      550
Humidity      550
Cloudiness    550
Wind Speed    550
Country       550
Date          550
dtype: int64
'''

r_data


NameError: name 'random' is not defined

In [ ]:
data = []
'''
here, I query open weather maps for each city, extracting the following:
name, latitude, longitude, daily high, humidity, cloud coverage %, wind speed
'''
print('Beginning Data Retrieval')     
print('-----------------------------')
j = 1
for i in range(len(cities)):
    params['q'] = cities[i]
    try:
        response = requests.get(base_url, params=params)
        r = response.json()
        city = r['name']
        lat = r['coord']['lat']
        lng = r['coord']['lon']
        max_temp = r['main']['temp_max']
        humid = r['main']['humidity']
        cloud = r['clouds']['all']
        wind = r['wind']['speed']
        country = r['sys']['country']
        today = date.today()

        r_data = [city,lat,lng,max_temp,humid,cloud,wind,country,today]
        
        data.append(r_data)
        print(f'Processing Record {i%50+1} of Set {j} | {cities[i]}')
        
    except (KeyError, IndexError):
        print('City not found. Skipping...')
        
    if (i + 1) % 50 == 0:
        j += 1
        time.sleep(10)
        

In [ ]:
data[0:5]

In [ ]:
for i in range(len(data)):
    data[i][8] = data[i][8].isoformat()

In [ ]:
'''
City          550
Lat           550
Lng           550
Max Temp      550
Humidity      550
Cloudiness    550
Wind Speed    550
Country       550
Date          550
dtype: int64
'''
weather_df = pd.DataFrame(data, columns=['City Name', 'Latitude', 'Longitude', 'Daily High', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date'])

In [ ]:
weather_df.head()

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_df.to_csv(output_data_file, index=False)

In [ ]:
# weather_df = pd.read_csv(output_data_file)
# weather_df.head()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
sweaty_df = weather_df.loc[weather_df['Humidity'] > 100]
sweaty_df.index

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = weather_df.drop(sweaty_df.index)
clean_city_data.head()

In [ ]:
# Extract relevant fields from the data frame
##Which fields?

# Export the City_Data into a csv
## a second csv?
output_data_file = 'output_data/clean_cities.csv'
clean_city_data.to_csv(output_data_file, index=False)

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

In [ ]:
def linRegPlot(x_values,y_values):
    

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression